In [6]:
%pip install pymongo

   ---------------------------------------- 0.0/483.7 kB ? eta -:--:--
   --- ----------------------------------- 41.0/483.7 kB 991.0 kB/s eta 0:00:01
   ----- --------------------------------- 71.7/483.7 kB 660.6 kB/s eta 0:00:01
   --------- ---------------------------- 122.9/483.7 kB 804.6 kB/s eta 0:00:01
   -------------- ----------------------- 184.3/483.7 kB 857.5 kB/s eta 0:00:01
   ----------------- -------------------- 225.3/483.7 kB 919.0 kB/s eta 0:00:01
   -------------------- ----------------- 256.0/483.7 kB 983.0 kB/s eta 0:00:01
   ------------------------ ------------- 317.4/483.7 kB 936.6 kB/s eta 0:00:01
   ---------------------------------- ----- 419.8/483.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 483.7/483.7 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   -------------------------- ------------- 204.8/307.7 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 307.7/307.7 kB

In [8]:
%pip list >> requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
from typing import List, Dict, Any
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
from pymongo import MongoClient

In [ ]:
class Case:
    def __init__(self, problem: Dict[str, Any], solution: Any):
        self.problem = problem
        self.solution = solution

In [ ]:
class CaseBase:
    # def __init__(self):
    #     self.cases: List[Case] = []
    def __init__(self, db_name: str, collection_name: str, uri: str = "mongodb://localhost:27017/"):
        self.client = MongoClient(uri)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

    def add_case(self, case: Case):
        # self.cases.append(case)
        self.collection.insert_one({"problem": case.problem, "solution": case.solution})
        
    def get_all_cases(self) -> List[Case]:
        cases = []
        for document in self.collection.find():
            case = Case(problem=document["problem"], solution=document["solution"])
            cases.append(case)
        return cases

    def find_most_similar_case(self, new_problem: Dict[str, Any]) -> Case:
        distances = []
        for case in self.cases:
            distances.append(self._calculate_distance(new_problem, case.problem))
        min_index = np.argmin(distances)
        return self.cases[min_index]

    def _calculate_distance(self, problem1: Dict[str, Any], problem2: Dict[str, Any]) -> float:
        vec1 = np.array(list(problem1.values())).reshape(1, -1)
        vec2 = np.array(list(problem2.values())).reshape(1, -1)
        return euclidean_distances(vec1, vec2)[0][0]


In [ ]:
# Example usage
if __name__ == "__main__":
    # Define some cases
    case1 = Case(problem={"feature1": 1.0, "feature2": 2.0}, solution="Solution 1")
    case2 = Case(problem={"feature1": 2.0, "feature2": 3.0}, solution="Solution 2")

    # Create a case base and add cases to it
    case_base = CaseBase()
    case_base.add_case(case1)
    case_base.add_case(case2)

    # Define a new problem
    new_problem = {"feature1": 1.5, "feature2": 2.5}

    # Retrieve the most similar case
    similar_case = case_base.find_most_similar_case(new_problem)

    # Print the retrieved case's solution
    print(f"The most similar case solution is: {similar_case.solution}")